
# CS301 Midterm Project — EDA & Visualization (YouTube Data)

**Student:** Elvis B. Otieno  
**Course:** CS301 (Data Science)  
**Project:** Exploratory Data Analysis & Visualization  
**Dataset:** `olaolatunde/data-for-youtube` (Kaggle)

> In this notebook I walk through my process: how I picked the dataset, how I cleaned it,
> what I decided to keep/drop, what I visualized, and what I learned. I kept the tone practical
> and focused on telling a clear data story.


In [ ]:
pip install kagglehub pandas matplotlib numpy


Defaulting to user installation because normal site-packages is not writeable
Looking in links: /usr/share/pip-wheels


In [ ]:

# 0) Setup
import os
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# I prefer readable numbers
pd.set_option("display.float_format", lambda x: f"{x:,.3f}")



## 1) Dataset Description (what it is and what I want to learn)

**Source.** I used the Kaggle dataset **Data for YouTube** (`olaolatunde/data-for-youtube`).
It contains video-level metrics like titles, views, likes, and comments. This is perfect for
a lightweight EDA because the columns are understandable and the questions are intuitive.

**My questions.**
1. Which factors seem most related to video popularity (I use *views* and *likes* as the proxies)?  
2. Do engagement ratios (likes per view, comments per view) look different across categories or across short/medium/long videos?

**Why this matters.** If I were a creator, I’d want to know which knobs are associated with more engagement,
and whether certain categories do better per view than others.



## 2) Getting the Data (Kaggle + fallback)

I first try to download via `kagglehub`. If that doesn’t work on your machine, you can
manually download the CSV from Kaggle and set `manual_csv_path` below.


In [ ]:

# 2.1) Try kagglehub, then search for CSVs in the cache
csv_candidates = []
dataset_path = None

try:
    import kagglehub
    dataset_path = kagglehub.dataset_download("olaolatunde/data-for-youtube")
    print("Kaggle dataset cached at:", dataset_path)
    for root, _, files in os.walk(dataset_path):
        for f in files:
            if f.lower().endswith(".csv"):
                csv_candidates.append(os.path.join(root, f))
except Exception as e:
    print("Kaggle download not available here:", e)

csv_candidates


In [ ]:

# 2.2) Fallback: set this to your local CSV path if Kaggle download is unavailable
manual_csv_path = "youtube_data.csv"  # update if you saved a local copy

def choose_csv(cands):
    if not cands:
        return None
    yt_like = [p for p in cands if "youtube" in os.path.basename(p).lower()]
    pool = yt_like if yt_like else cands
    # pick the largest file as a simple heuristic
    return max(pool, key=lambda p: os.path.getsize(p))

csv_path = choose_csv(csv_candidates)

if (csv_path is None) and os.path.exists(manual_csv_path):
    csv_path = manual_csv_path

print("Using CSV:", csv_path)
if csv_path is None:
    raise FileNotFoundError("No CSV found yet. Set manual_csv_path to a valid local file.")



## 3) First look at the raw data

Here I load the CSV, check shape and types, and preview a few rows. This helps me decide
what columns matter and where the messy parts are (missingness, text vs numeric, etc.).


In [ ]:

import os, csv
from itertools import islice

print("Reading:", csv_path, "| size:", os.path.getsize(csv_path), "bytes")

def robust_read_csv(path):
    # Try to sniff delimiter first
    with open(path, "r", encoding="utf-8", errors="replace") as f:
        sample = "".join(list(islice(f, 200)))
    try:
        dialect = csv.Sniffer().sniff(sample, delimiters=[",",";","|","\t"])
        sep = dialect.delimiter
    except Exception:
        sep = None  # let pandas infer

    # Most forgiving parse first
    try:
        df0 = pd.read_csv(
            path,
            sep=sep,                # None -> infer; else use sniffed
            engine="python",        # more tolerant than C engine
            encoding="utf-8",
            encoding_errors="replace",
            on_bad_lines="skip",    # skip malformed records
        )
        return df0
    except Exception as e1:
        print("First pass failed:", e1)

    # Try common alt encodings / delimiters
    for enc in ["utf-8-sig", "latin-1"]:
        for alt_sep in [sep, ",", ";", "\t", "|", None]:
            try:
                df0 = pd.read_csv(
                    path,
                    sep=alt_sep,
                    engine="python",
                    encoding=enc,
                    encoding_errors="replace",
                    on_bad_lines="skip",
                )
                print(f"Loaded with encoding={enc}, sep={alt_sep!r}")
                return df0
            except Exception as e2:
                last_err = e2
    raise last_err

df_raw = robust_read_csv(csv_path)
df = df_raw.copy()
print("Loaded shape:", df.shape)
df.head()



# User
yes


## 4) Cleaning & preparation (what I changed and why)

What I chose to do:
- **Dedup** rows so each video appears once.
- **Standardize** column names to snake_case (less typing errors later).
- **Coerce types**: make obvious numeric columns numeric and parse dates where it makes sense.
- **Missing values**: I drop rows missing *key* numeric metrics (views/likes/comments) since imputing
  those would make the ratios unreliable. Categorical nulls I fill with `"Unknown"` to keep rows.
- **Feature engineering**: `like_rate = likes / views`, `comment_rate = comments / views`.
  If there’s a duration column, I bucket it into Short/Medium/Long (terciles) to compare groups.


In [ ]:

# 4.1) Deduplicate
before = len(df)
df = df.drop_duplicates()
print(f"Deduplicated: {before - len(df)} rows removed (now {len(df)})")

# 4.2) Standardize column names
df.columns = [c.strip().lower().replace(" ", "_") for c in df.columns]

# 4.3) Identify columns by name patterns (flexible across dataset versions)
num_hints  = ["view", "like", "comment", "duration", "dislike", "share"]
cat_hints  = ["category", "channel", "title"]
date_hints = ["date", "published"]

likely_numeric = [c for c in df.columns if any(k in c for k in num_hints)]
likely_cats    = [c for c in df.columns if any(k in c for k in cat_hints)]
likely_dates   = [c for c in df.columns if any(k in c for k in date_hints)]

print("Likely numeric:", likely_numeric)
print("Likely categorical:", likely_cats)
print("Likely datetime:", likely_dates)

# 4.4) Convert types
for c in likely_dates:
    try:
        df[c] = pd.to_datetime(df[c], errors="coerce")
    except Exception:
        pass

for c in likely_numeric:
    df[c] = pd.to_numeric(df[c], errors="coerce")

# 4.5) Missingness report
missing_report = df.isna().mean().sort_values(ascending=False)
missing_report.head(12)


In [ ]:

# 4.6) Handle missing values
central_numeric = [c for c in ["views","likes","comments"] if c in df.columns]

for c in central_numeric:
    before = len(df)
    df = df[~df[c].isna()]
    print(f"Dropped {before - len(df)} rows due to missing {c}. (Now {len(df)})")

for c in df.select_dtypes(include="object").columns:
    if df[c].isna().any():
        df[c] = df[c].fillna("Unknown")

# 4.7) Light outlier capping so charts aren't dominated by a few massive points
for c in central_numeric:
    if c in df.columns:
        cap = df[c].quantile(0.999)
        df[c] = np.minimum(df[c], cap)

print("Cleaning complete.")


In [ ]:

# 4.8) Feature engineering
if "views" in df.columns:
    if "likes" in df.columns:
        df["like_rate"] = df["likes"] / df["views"].replace(0, np.nan)
    if "comments" in df.columns:
        df["comment_rate"] = df["comments"] / df["views"].replace(0, np.nan)

# duration buckets if any duration-like column is present
dur_col = None
for c in df.columns:
    if "duration" in c:
        dur_col = c
        break

if dur_col is not None:
    if df[dur_col].dtype == object:
        df[dur_col] = pd.to_numeric(df[dur_col], errors="coerce")
    if df[dur_col].notna().sum() > 0:
        q = df[dur_col].quantile([0.33, 0.66]).values
        def bucket(x):
            if pd.isna(x): return "Unknown"
            if x <= q[0]: return "Short"
            if x <= q[1]: return "Medium"
            return "Long"
        df["duration_bucket"] = df[dur_col].apply(bucket)

df.head(8)



## 5) EDA — What does the data say at a glance?

I start with simple descriptives and a correlation matrix. My expectation is that views/likes/comments
all move together (exposure → engagement), but engagement *rates* might separate by category/duration.


In [ ]:

desc = df.describe(include="all")




In [ ]:

num_df = df.select_dtypes(include=[np.number])
corr = num_df.corr(numeric_only=True)
corr



## 6) Visualizations (plain matplotlib, one chart per figure)

> Per my class guidelines, I used **matplotlib only**, **no custom colors**, and **one plot per figure**.



### 6.1) Histogram — Views
I expected a long right tail (viral videos). The histogram helps confirm that most videos are modest, a few are huge.


In [ ]:

if "views" in df.columns:
    plt.figure()
    df["views"].dropna().plot(kind="hist", bins=50)
    plt.title("Distribution of Views")
    plt.xlabel("Views")
    plt.ylabel("Frequency")
    plt.show()
else:
    print("Skipping: 'views' column not found.")



### 6.2) Scatter — Likes vs Views (log scales)
If views and likes scale together, a diagonal cloud should appear on a log–log scatter.


In [ ]:

if all(c in df.columns for c in ["views","likes"]):
    plt.figure()
    plt.scatter(df["views"], df["likes"], s=9, alpha=0.5)
    plt.xscale("log")
    plt.yscale("log")
    plt.title("Likes vs Views (log–log)")
    plt.xlabel("Views")
    plt.ylabel("Likes")
    plt.show()
else:
    print("Skipping: need both 'views' and 'likes'.")



### 6.3) Boxplot — Like Rate by Category
I wanted to see if some categories punch above their weight (higher likes per view). If the dataset has a category column, this shows it.


In [ ]:

cat_col = None
for c in df.select_dtypes(include=["object"]).columns:
    if "category" in c:
        cat_col = c
        break

if cat_col is not None and "like_rate" in df.columns:
    groups, labels = [], []
    for k, g in df[[cat_col, "like_rate"]].dropna().groupby(cat_col):
        vals = g["like_rate"].values
        if len(vals) > 0:
            groups.append(vals)
            labels.append(str(k)[:18])

    if groups:
        plt.figure()
        plt.boxplot(groups, showfliers=False)
        plt.title("Like Rate by Category")
        plt.xlabel("Category")
        plt.ylabel("Likes / Views")
        plt.xticks(range(1, len(labels)+1), labels, rotation=45, ha="right")
        plt.tight_layout()
        plt.show()
    else:
        print("No non-empty groups for category vs like_rate.")
else:
    print("Skipping: category and/or like_rate not available.")



### 6.4) Heatmap — Numeric Correlations
This is a quick overview of which numeric features move together.


In [ ]:

if 'corr' in globals() and corr.size > 0:
    plt.figure()
    im = plt.imshow(corr.values, aspect="auto")
    plt.title("Correlation Heatmap (Numeric)")
    plt.xlabel("Features")
    plt.ylabel("Features")
    plt.xticks(range(len(corr.columns)), corr.columns, rotation=45, ha="right")
    plt.yticks(range(len(corr.index)), corr.index)
    plt.colorbar(im, fraction=0.046, pad=0.04)
    plt.tight_layout()
    plt.show()
else:
    print("Skipping heatmap: no numeric correlation matrix.")



## 7) What I learned (and what I'd try next)

**Takeaways.**
- **Exposure drives engagement:** Views and likes typically rise together (strong positive correlation),
  which matches my expectations for platform dynamics.
- **Efficiency differs by content:** Like-per-view (and often comment-per-view) varies by category and
  sometimes by duration bucket. Some topics attract more *per-view* enthusiasm than others.
- **Heavy right tail:** A small fraction of videos dominate total engagement, so I capped extreme outliers
  slightly to keep plots readable.

**If I had more time**, I’d add a simple regression with log(views) as the target to see which features
hold up after controlling for others, and I’d check robustness with different outlier rules.
